# Importing Packges

In [1]:
%matplotlib inline

import numpy as np

import joblib


import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score, classification_report, confusion_matrix
import collections
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

np.random.seed(1337)  # for reproducibility



from sklearn.ensemble import VotingClassifier

# Loading data (CTTD features and Labels)

In [19]:
X_train = np.load('../data/S10/X_train.npy') 

Y_train = np.load('../data/S10/Y_train.npy')

X_test = np.load('../data/S10/X_test.npy')

Y_test = np.load('../data/S10/Y_test.npy')

X_test2 = np.load('../data/S10/X_test2.npy')

Y_test2 = np.load('../data/S10/Y_test2.npy')

In [22]:
np.std(X_test2, axis=0)

array([0.00736808, 0.01305474, 0.01328746, 0.01087264, 0.00999119,
       0.01113731, 0.00998801, 0.01845082, 0.02114146, 0.02708462,
       0.02041779, 0.01724392, 0.01643513, 0.04121975, 0.01546927,
       0.07796691, 0.04419348, 0.04710152, 0.06742214, 0.09496654,
       0.0495976 , 0.02998133, 0.04732659, 0.04302824, 0.04004036,
       0.04152549, 0.04763844, 0.03762704, 0.00567291, 0.00851655,
       0.00403908, 0.00333027, 0.00517552, 0.0102636 , 0.01745613,
       0.26969236, 0.35434232, 0.36674884, 0.30872286, 0.21107681,
       0.24631183, 0.30003703])

# Scaling features

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_test2 = scaler.fit_transform(X_test2)

In [16]:
np.std(X_test2, axis=0)

array([0.00832342, 0.01271719, 0.01000976, 0.00798374, 0.0092489 ,
       0.01024944, 0.00596773, 0.01591231, 0.01564641, 0.01319807,
       0.01187793, 0.01156039, 0.01127222, 0.01167754, 0.03347382,
       0.0920087 , 0.0598841 , 0.04568769, 0.08978898, 0.08957959,
       0.02143906, 0.03788116, 0.05597681, 0.04140564, 0.04269216,
       0.04195512, 0.05372792, 0.0312801 , 0.00610192, 0.01203334,
       0.00880538, 0.00864165, 0.00787116, 0.01205243, 0.00676343,
       0.28187852, 0.34139805, 0.31967943, 0.30485763, 0.23461088,
       0.28609784, 0.24433569])

In [4]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)


print(X_test2.shape)
print(Y_test2.shape)

(28368, 42)
(28368,)
(12168, 42)
(12168,)
(7040, 42)
(7040,)


# Loading models 

In [10]:
SVM_clf = joblib.load('../models/svm_emg_clf.pkl')

LR_clf = joblib.load('../models/LR_emg_clf.pkl')

RF_clf = joblib.load('../models/RF_emg_clf.pkl')

ET_clf = joblib.load('../models/ET_emg_clf.pkl')

C:\Users\Suman\anaconda3\envs\sumanEnv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Suman\anaconda3\envs\sumanEnv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Suman\anaconda3\envs\sumanEnv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ExtraTreeClassifier from version 0.22.2.post1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\Suman\anaconda3\envs\sumanEnv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ExtraTreesClassifie

In [7]:
estimators=[('LR',LR_clf),('SVM',SVM_clf),('RF',RF_clf),('ET',ET_clf)]

In [8]:
voting_clf=VotingClassifier(estimators,voting='soft')

In [9]:
voting_clf.fit(X_train,Y_train)

VotingClassifier(estimators=[('LR',
                              LogisticRegression(C=100, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=10000,
                                                 multi_class='multinomial',
                                                 n_jobs=-1, penalty='l2',
                                                 random_state=None,
                                                 solver='saga', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('SVM',
                              SVC(C=35.4, break_ties=False, cache_size=200,
                                  class_weight='balanced...
                                                   criterion='gini',
                                          

In [10]:
Y_predict = voting_clf.predict(X_test)

In [11]:
accuracy_score(Y_test, Y_predict)

0.9326101249178173

# SVC class don't compute probability by default, so we have to train the model by setting the parameter probability=True.

In [13]:
SVM_clf = SVC(kernel = 'rbf', C = 35.4, gamma= 'scale', class_weight = 'balanced',probability=True)

In [ ]:
SVM_clf.fit(X_train, Y_train)

# Evaluation on Test set 1

In [15]:
for clf in (LR_clf,SVM_clf,RF_clf,ET_clf,voting_clf):
    y_predict=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(Y_test, y_predict)) 

LogisticRegression 0.9178994082840237
SVC 0.9231591058514136
RandomForestClassifier 0.9059007232084155
ExtraTreesClassifier 0.9275147928994083
VotingClassifier 0.9326101249178173


# Evaluation on Test set 2

In [16]:
for clf in (LR_clf,SVM_clf,RF_clf,ET_clf,voting_clf):
    y_predict=clf.predict(X_test2)
    print(clf.__class__.__name__,accuracy_score(Y_test2, y_predict)) 

LogisticRegression 0.9676136363636364
SVC 0.9762784090909091
RandomForestClassifier 0.953125
ExtraTreesClassifier 0.9627840909090909
VotingClassifier 0.9738636363636364
